# Benchmarking

In [15]:
import sys
import time
import os
import numpy as np
import pyswarms as ps
from skopt import gp_minimize
from skopt.space import Real, Space
from skopt.utils import use_named_args
# sys.path.append('/Users/calum/Developer/pyzx-heuristics-master')
# import pyzx as zx_heuristics
sys.path.append('..')
import pyzx as zx
from benchmarking import benchmark

In [2]:
# %%script false --no-raise-error
b = benchmark(dirpath='benchmark_gen')
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_simp', 't_opt']
Loaded functions:  ['Basic', 'cFlow', 'gFlow', 'Heur']
Loaded routines:  []
Loaded circuit groups:  ['generated']


,Original,Basic,Heur,cFlow,gFlow
generated,Y,Y,Y,Y,Y


In [3]:
%%script false --no-raise-error
b = benchmark()
b.load_circuits('generated_circuits', group_name='generated')

In [11]:
base_params = [1,1,1,0,0,0,0,0,0]
cFlow_params = [0.37,0.4,0.23,-0.94,-0.71,0.5,-0.45,0.47,0.59]
pp = [1,1,1,0,0,0,0,0,99]

def cFlow_reduce(c):
    t0 = time.time()
    g = c.to_graph()
    zx.simplify.to_gh(g)
    g2 = zx.simplify.flow_reduce(g,x=pp, quiet=True)
    t1 = time.time()
    c2 = zx.extract.extract_simple(g2, up_to_perm=True).to_basic_gates()
    c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t1-t0, t2-t0
    return c3, t1-t0, t2-t0

def gFlow_reduce(c):
    t0 = time.time()
    g = c.to_graph()
    zx.simplify.to_gh(g)
    g2 = zx.simplify.flow_reduce(g,x=base_params, flow = 'g', quiet=True)
    t1 = time.time()
    c2 = zx.extract.extract_circuit(g2, up_to_perm=True).to_basic_gates()
    c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t1-t0, t2-t0
    return c3, t1-t0, t2-t0

def zx_heur(c):
    t0 = time.time()
    g = c.to_graph()
    g = zx_heuristics.simplify.teleport_reduce(g)
    g.track_phases = False
    zx_heuristics.simplify.greedy_simp_neighbors(g)
    t1 = time.time()
    c2 = zx_heuristics.extract_circuit(g, up_to_perm=True).to_basic_gates()
    c3 = zx_heuristics.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx_heuristics.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t1-t0, t2-t0
    return c3, t1-t0, t2-t0

def basic(c):
    c1 = zx.optimize.basic_optimization(c.copy(), do_swaps=False).to_basic_gates()
    c2 = zx.optimize.basic_optimization(c.copy(), do_swaps=True).to_basic_gates()
    if c2.twoqubitcount() < c1.twoqubitcount(): return c2
    return c1

In [12]:
# b.add_simplification_func(zx_heur, 'Heur', groups_to_run=['generated'],verify=False)
# b.add_simplification_func(basic, 'Basic', groups_to_run=['generated'],verify=False)
b.add_simplification_func(cFlow_reduce, 'cFlow3', groups_to_run=['generated'],verify=False,rerun=True)
# b.add_simplification_func(gFlow_reduce, 'gFlow', groups_to_run=['generated'],verify=False)

Processing cFlow3 on c_17                                             : 100%|██████████| 20/20 [00:10<00:00,  1.95it/s]


In [17]:
for filename in os.listdir('generated_circuits'):
    f = os.path.join('generated_circuits', filename)
    print(f)

generated_circuits/c_10.qc
generated_circuits/c_14.qc
generated_circuits/c_15.qc
generated_circuits/c_11.qc
generated_circuits/c_0.qc
generated_circuits/c_4.qc
generated_circuits/c_5.qc
generated_circuits/c_1.qc
generated_circuits/c_6.qc
generated_circuits/c_18.qc
generated_circuits/c_2.qc
generated_circuits/c_19.qc
generated_circuits/c_3.qc
generated_circuits/c_7.qc
generated_circuits/c_16.qc
generated_circuits/c_12.qc
generated_circuits/c_8.qc
generated_circuits/c_13.qc
generated_circuits/c_9.qc
generated_circuits/c_17.qc


In [19]:
c = zx.Circuit.load('generated_circuits/c_10.qc').to_basic_gates()

def cFlow_reduce(x):
    res = []
    for xi in x:
        xi = xi / np.sum(xi)
        g = c.to_graph()
        zx.simplify.to_gh(g)
        g2 = zx.simplify.flow_reduce(g,x=xi, quiet=True)
        c2 = zx.extract.extract_simple(g2, up_to_perm=True).to_basic_gates()
        c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates().twoqubitcount()
        c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates().twoqubitcount()
        result = min(c3, c4)
        res.append(result)
    return res

lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])
upper_bounds = np.array([10, 10, 10, 10, 10, 10, 10, 10, 10]) 
bounds = (lower_bounds, upper_bounds)

def optimise(params):
    c1, c2, w = params
    options = {'c1': c1, 'c2': c2, 'w': w}  # Adjust 'c1' to be higher than 'c2' for more exploration.
    
    costs = []
    for filename in os.listdir('generated_circuits'):
        f = os.path.join('generated_circuits', filename)
        c = zx.Circuit.load(f).to_basic_gates()
        def cFlow_reduce2(x):
            res = []
            for xi in x:
                xi = xi / np.sum(xi)
                g = c.to_graph()
                zx.simplify.to_gh(g)
                g2 = zx.simplify.flow_reduce(g,x=xi, quiet=True)
                c2 = zx.extract.extract_simple(g2, up_to_perm=True).to_basic_gates()
                c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates().twoqubitcount()
                c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates().twoqubitcount()
                result = min(c3, c4)
                res.append(result)
            return res 
        optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=9, options=options, bounds=bounds)
        cost, pos = optimizer.optimize(cFlow_reduce, iters=100)
        costs.append(cost)
    return np.sum(costs)

# print(optimise((1.5,1,0.9)))

space  = [Real(0.1, 1.5, name='c1'),
          Real(0.1, 1.5, name='c2'),
          Real(0.1, 1.0, name='w')]

res_gp = gp_minimize(optimise, space, n_calls=50, random_state=0)

print("Best parameters: c1=%.4f, c2=%.4f, w=%.4f" % (res_gp.x[0], res_gp.x[1], res_gp.x[2]))

2023-07-16 20:17:36,105 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.9299824655150257, 'c2': 1.2819720480134247, 'w': 0.8721510558604813}
pyswarms.single.global_best:  62%|██████▏   |62/100, best_cost=15


KeyboardInterrupt: 

In [11]:
def cFlow_swarm(c):
    
    def cFlow_reduce(x):
        g = c.to_graph()
        zx.simplify.to_gh(g)
        g2 = zx.simplify.flow_reduce(g,x=base_params, quiet=True)
        c2 = zx.extract.extract_simple(g2, up_to_perm=True).to_basic_gates()
        c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates().twoqubitcount()
        c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates().twoqubitcount()
        return min(c3, c4)

    def optimise():
        options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

        optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=9, options=options)
        cost, pos = optimizer.optimize(cFlow_reduce, iters=1000)
        
        return pos
    
    t0 = time.time()
    pos = optimise()
    g = c.to_graph()
    zx.simplify.to_gh(g)
    g2 = zx.simplify.flow_reduce(g,x=pos, quiet=True)
    t1 = time.time()
    c2 = zx.extract.extract_simple(g2, up_to_perm=True).to_basic_gates()
    c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t1-t0, t2-t0
    return c3, t1-t0, t2-t0

In [12]:
b.add_simplification_func(cFlow_swarm, 'cFlow_swarm', groups_to_run=['generated'],verify=False)

Processing cFlow_swarm on c_10                                        :   0%|          | 0/20 [00:00<?, ?it/s]2023-07-16 18:52:22,059 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=15
2023-07-16 18:54:12,417 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 15.0, best pos: [0.9840272  0.33286694 0.99074681 0.12477358 0.08102259 0.93531074
 0.30589334 0.3181413  0.2448119 ]
Processing cFlow_swarm on c_14                                        :   5%|▌         | 1/20 [01:50<34:58, 110.47s/it]2023-07-16 18:54:12,493 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:  14%|█▍        |140/1000, best_cost=98
Processing cFlow_swarm on c_14                                        :   5%|▌         | 1/20 [04:51<1:32:23, 291.77s/it]


KeyboardInterrupt: 

In [13]:
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_simp', 't_opt']
Loaded functions:  ['Basic', 'cFlow', 'gFlow', 'Heur', 'cFlow2', 'cFlow3']
Loaded routines:  []
Loaded circuit groups:  ['generated']


,Original,Basic,Heur,cFlow,cFlow2,cFlow3,gFlow
generated,Y,Y,Y,Y,Y,Y,Y


In [14]:
df = b.df(groups=['generated'],
     routines='all',
     funcs='all',
     atts=['Qubits', '2Q Count', 'T Count', 't_opt'])

In [8]:
b.save(dirpath='benchmark_gen')